# AWS Python SDK(boto3) DataZone Client의 Search 메서드를 이용하여 Asset 메타데이터 저장

- 아래 **현재 실행 중인 IAM 정보**는 실제 호출할 주체를 확인하기 위해 실행



In [1]:
import boto3

sts = boto3.client("sts")
identity = sts.get_caller_identity()

print("✅ 현재 실행 중인 IAM Identity 정보:")
print(f"Account ID : {identity['Account']}")
print(f"UserId     : {identity['UserId']}")
print(f"Arn        : {identity['Arn']}")


✅ 현재 실행 중인 IAM Identity 정보:
Account ID : 533616270150
UserId     : AIDAXYPQCDNDD6RMCGL25
Arn        : arn:aws:iam::533616270150:user/mj.kwon@sk.com


### Asset 메타데이터 전처리 과정을 **SMUS Notebook** 내에서 실행할 경우에는 아래와 같이 출력됨.

✅ **현재 실행 중인 IAM Identity 정보**

| 항목 | 값 |
|------|----|
| Account ID | `533616270150` |
| UserId | `AROAXYPQCDNDMUFTA7JFO:SageMaker` |
| Arn | `arn:aws:sts::533616270150:assumed-role/datazone_usr_role_bknisgxzopjuhk_4oa8qs5krhk8ig/SageMaker` |

---

### SMUS Notebook에서 실행할 경우 아래 정책을 **`datazone_usr_role_bknisgxzopjuhk_4oa8qs5krhk8ig`** 역할에 추가해야 함:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowDataZoneReadAccess",
      "Effect": "Allow",
      "Action": [
        "datazone:Search",
        "datazone:GetAsset",
        "datazone:GetDomain",
        "datazone:ListProjects",
        "datazone:GetProject"
      ],
      "Resource": [
        "arn:aws:datazone:ap-northeast-2:533616270150:domain/dzd-cjvglgj4d43fmg",
        "arn:aws:datazone:ap-northeast-2:533616270150:domain/dzd-cjvglgj4d43fmg/*"
      ]
    }
  ]
}


### SMUS Domain 내 전체 Asset을 조회
- AWS SDK DataZone Client의 Search메서드를 사용하며, 아래 자세한 정보 확인 가능
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/datazone/client/search.html
- Search메서드는 searchScope 기준으로 조회가 가능하며, 아래와 같이 4가지를 지원
- searchScope: "ASSET" || "GLOSSARY" || "GLOSSARY_TERM" || "DATA_PRODUCT"

### 아래의 코드는 전체 Asset 중 GlueTableAssetType + listing_status == "ACTIVE" (PUBLISHED)만 찾아서 해당 Asset 별 메타데이터를 "[Asset명]_[AssetID].json"형식으로 저장하는 예시 

# 초기적재, 변경분 반영 코드 분리

## 초기적재 코드

In [40]:
import boto3
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor 
from datetime import datetime, timezone 

# --- 환경 설정 ---
region = "ap-northeast-2"
dz = boto3.client("datazone", region_name=region)
domain_id = "dzd-3ojk7mnm02q5lk"
MAX_THREADS = 50 

# 💡 KB Data Source가 바라볼 최종 동기화 폴더 (전체/델타 공용)
KB_S3_SYNC_DIR = "asset_result_data/glue_assets_kb_sync_final" 
os.makedirs(KB_S3_SYNC_DIR, exist_ok=True)
PUBLISHED_ASSET_SNAPSHOT_FILE = "published_asset_update_snapshot_initial.json" # 스냅샷 파일명


# ----------------------------------------------------------------------
# 💡 헬퍼 함수 및 Step 3 함수 정의
# ----------------------------------------------------------------------

def format_time_to_iso(dt_object):
    """datetime 객체를 KB 관리에 필요한 UTC ISO 문자열로 변환합니다."""
    if dt_object and isinstance(dt_object, datetime):
        return dt_object.astimezone(timezone.utc).isoformat().replace('+00:00', 'Z')
    if isinstance(dt_object, str):
        return dt_object.replace('+00:00', 'Z') if '+00:00' in dt_object else dt_object
    return None

def fetch_and_filter_asset(asset_info):
    """상세 정보를 가져와 상태 확인 및 고유 키를 생성합니다."""
    asset_id = asset_info["id"]
    
    try:
        asset_resp = dz.get_asset(domainIdentifier=domain_id, identifier=asset_id)
        
        # 1. GlueTableForm JSON 문자열 추출 및 파싱
        glue_form_content = next((item['content'] for item in asset_resp.get('formsOutput', []) 
                                  if item['formName'] == 'GlueTableForm'), None)
        
        if not glue_form_content: return None
        glue_form_data = json.loads(glue_form_content)

        # 2. 고유 키 생성
        db_name = glue_form_data.get("databaseIdentifier") or glue_form_data.get("databaseName")
        table_name = glue_form_data.get("tableName")
        if not db_name or not table_name: return None
        unique_key = f"{db_name}_{table_name}"
        
        # 🚀 [최종 수정] 3. 최종 업데이트 시간 (Asset Revision 기반) 추출
        # Asset 메타데이터가 수정된 시간: Boto3는 이 필드를 'createdAt'으로 반환
        current_update_time_dt = asset_resp.get("createdAt") 
        current_time_str = format_time_to_iso(current_update_time_dt) # ISO 문자열로 변환
        
        # 4. 게시 상태 확인
        listing_status = (asset_resp.get("listing", {}) or {}).get("listingStatus", "UNKNOWN")
        
        if listing_status == "ACTIVE":
            asset_info.update({
                "unique_key": unique_key, 
                # 🚀 [수정 적용] 추출한 문자열을 그대로 저장
                "current_update_time": current_time_str, # Asset Revision 시간 저장
                "metadata": asset_resp
            })
            return asset_info
            
        return None

    except Exception as e:
        # print(f"❌ Error fetching asset details for {asset_id}: {e}") # 디버그용
        return None


# ----------------------------------------------------------------------
# 2️⃣ 모든 GlueTableAssetType의 ID 수집 (전체 조회)
# ----------------------------------------------------------------------
projects_resp = dz.list_projects(domainIdentifier=domain_id)
projects = {p["id"]: p["name"] for p in projects_resp.get("items", [])}
all_glue_asset_ids = []

for project_id, project_name in projects.items():
    next_token = None
    while True:
        params = {"domainIdentifier": domain_id, "owningProjectIdentifier": project_id, "searchScope": "ASSET"}
        if next_token: params["nextToken"] = next_token

        resp = dz.search(**params)
        for item in resp.get("items", []):
            if item.get("assetItem", {}).get("typeIdentifier") == "amazon.datazone.GlueTableAssetType":
                all_glue_asset_ids.append({"id": item.get("assetItem", {}).get("identifier"), "name": item.get("assetItem", {}).get("name")})
        next_token = resp.get("nextToken")
        if not next_token: break

print(f"✅ Total {len(all_glue_asset_ids)} GlueTableAssetType assets found by search. Starting detail fetch...")

# ----------------------------------------------------------------------
# 3️⃣ get_asset 호출 및 최종 필터링 (병렬 처리 실행)
# ----------------------------------------------------------------------
published_assets = []
MAX_THREADS = 50 

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    results = executor.map(fetch_and_filter_asset, all_glue_asset_ids)
    for asset in results:
        if asset:
            published_assets.append(asset)

print(f"\n✅ Final Count: {len(published_assets)} PUBLISHED assets ready for initial load.")

# ----------------------------------------------------------------------
# 4️⃣ JSON 저장 및 스냅샷 생성 (초기 적재)
# ----------------------------------------------------------------------
current_update_snapshot = {}

print("\n--- Step 4: Saving ALL JSON files and creating initial snapshot ---")

for asset in published_assets:
    unique_key = asset["unique_key"]
    output_path = os.path.join(KB_S3_SYNC_DIR, f"{unique_key}.json") 
    update_dt = asset["current_update_time"] # Asset Revision 시간

    # 4-A. JSON 파일 저장
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(
            {"unique_key": unique_key, "metadata": asset["metadata"], "updatedAt": update_dt}, 
            f, default=str, ensure_ascii=False, indent=2
        )
    
    current_update_snapshot[unique_key] = update_dt
    print(f"✅ Saved Asset: {output_path}")

# 4-C. 다음 일 배치를 위해 스냅샷 파일 저장
with open(PUBLISHED_ASSET_SNAPSHOT_FILE, "w") as f:
    json.dump(current_update_snapshot, f, indent=2)

print(f"\n🎉 Initial Full Load Complete. ({len(current_update_snapshot)} assets saved)")

✅ Total 50 GlueTableAssetType assets found by search. Starting detail fetch...

✅ Final Count: 2 PUBLISHED assets ready for initial load.

--- Step 4: Saving ALL JSON files and creating initial snapshot ---
✅ Saved Asset: asset_result_data/glue_assets_kb_sync_final_test/cloudtrail_logs_s3_trail_events.json
✅ Saved Asset: asset_result_data/glue_assets_kb_sync_final_test/glue_db_cwkk5edf43ynmg_store_details.json

🎉 Initial Full Load Complete. (2 assets saved)


## 변경분 반영

In [44]:
import boto3
import json
import os
import re
from concurrent.futures import ThreadPoolExecutor 
from datetime import datetime, timedelta, timezone 

# --- 환경 설정 ---
region = "ap-northeast-2"
dz = boto3.client("datazone", region_name=region)
domain_id = "dzd-3ojk7mnm02q5lk"
MAX_THREADS = 50 

# 💡 KB 동기화 경로 및 스냅샷 파일 (초기 적재 스크립트와 동일해야 함)
KB_S3_SYNC_DIR = "asset_result_data/glue_assets_kb_sync_final" 
PUBLISHED_ASSET_SNAPSHOT_FILE = "published_asset_update_snapshot_initial.json" 


# ----------------------------------------------------------------------
# 💡 헬퍼 함수 정의
# ----------------------------------------------------------------------

def time_str_to_datetime(time_str):
    """ISO 8601 문자열을 UTC datetime 객체로 변환합니다."""
    if not time_str or time_str == 'None': return datetime.min.replace(tzinfo=timezone.utc)
    if isinstance(time_str, datetime): dt = time_str
    elif isinstance(time_str, str):
        time_str = time_str.replace('Z', '+00:00') if time_str.endswith('Z') else time_str
        try: dt = datetime.fromisoformat(time_str)
        except ValueError: dt = datetime.fromisoformat(time_str).replace(tzinfo=timezone.utc)
    else: return datetime.min.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def is_time_newer(current_time_str, previous_time_str):
    """현재 시간이 이전 저장 시간보다 최신인지 비교합니다."""
    current_dt = time_str_to_datetime(current_time_str)
    previous_dt = time_str_to_datetime(previous_time_str)
    return current_dt > previous_dt

def format_time_to_iso(dt_object):
    """datetime 객체를 KB 관리에 필요한 UTC ISO 문자열로 변환합니다."""
    if dt_object and isinstance(dt_object, datetime):
        return dt_object.astimezone(timezone.utc).isoformat().replace('+00:00', 'Z')
    if isinstance(dt_object, str):
        return dt_object.replace('+00:00', 'Z') if '+00:00' in dt_object else dt_object
    return None


def fetch_and_filter_asset(asset_info):
    """상세 정보를 가져와 상태 확인 및 고유 키를 생성합니다."""
    asset_id = asset_info["id"]
    try:
        # Boto3 클라이언트 세션은 안정성을 위해 스레드 내부에서 생성하는 것이 좋으나, 
        # 메인 스크립트와 동일한 dz 클라이언트 사용을 유지함. (권장 사항: 스레드 내부에서 클라이언트 재생성)
        asset_resp = dz.get_asset(domainIdentifier=domain_id, identifier=asset_id)
        
        # 1. GlueTableForm JSON 문자열 추출 및 파싱
        glue_form_content = next((item['content'] for item in asset_resp.get('formsOutput', []) if item['formName'] == 'GlueTableForm'), None)
        if not glue_form_content: return None
        glue_form_data = json.loads(glue_form_content)
        
        # 2. 고유 키 추출
        db_name = glue_form_data.get("databaseIdentifier") or glue_form_data.get("databaseName")
        table_name = glue_form_data.get("tableName")
        if not db_name or not table_name: return None
        unique_key = f"{db_name}_{table_name}"
        
        # 🚀 [최종 수정] 3. 최종 업데이트 시간 (Asset Revision 기반) 추출
        current_update_time_dt = asset_resp.get("createdAt") 
        current_time_str = format_time_to_iso(current_update_time_dt)
        
        # 4. 게시 상태 확인
        listing_status = (asset_resp.get("listing", {}) or {}).get("listingStatus", "UNKNOWN")
        
        if listing_status == "ACTIVE":
            asset_info.update({"unique_key": unique_key, "current_update_time": current_time_str, "metadata": asset_resp})
            return asset_info
        return None
        
    except Exception as e:
        if "ResourceNotFoundException" in str(e): 
            # Asset이 완전히 삭제된 경우, 삭제 감지를 위해 ID 반환
            return {"id": asset_id, "status": "DELETED"}
        return None


# ----------------------------------------------------------------------
# 💡 Step 0: 이전 스냅샷 로드 및 초기화
# ----------------------------------------------------------------------
previous_update_times = {} 
if os.path.exists(PUBLISHED_ASSET_SNAPSHOT_FILE):
    with open(PUBLISHED_ASSET_SNAPSHOT_FILE, 'r') as f:
        previous_update_times = json.load(f)
    print(f"Loaded {len(previous_update_times)} update records for delta comparison.")
else:
    print("⚠️ Warning: Snapshot file not found. Running initial load logic.", flush=True)


# ----------------------------------------------------------------------
# 2️⃣ 모든 GlueTableAssetType의 ID 수집 (전체 조회 - 델타 체크를 위함)
# ----------------------------------------------------------------------
projects_resp = dz.list_projects(domainIdentifier=domain_id)
projects = {p["id"]: p["name"] for p in projects_resp.get("items", [])}
all_glue_asset_ids = []

print("\n--- Step 2: Collecting ALL Asset IDs for Delta Check ---")

for project_id, project_name in projects.items():
    next_token = None
    while True:
        params = {"domainIdentifier": domain_id, "owningProjectIdentifier": project_id, "searchScope": "ASSET"}
        if next_token: params["nextToken"] = next_token

        resp = dz.search(**params)
        for item in resp.get("items", []):
            if item.get("assetItem", {}).get("typeIdentifier") == "amazon.datazone.GlueTableAssetType":
                all_glue_asset_ids.append({"id": item.get("assetItem", {}).get("identifier")})
        next_token = resp.get("nextToken")
        if not next_token: break

print(f"✅ Total {len(all_glue_asset_ids)} assets found for detailed check.")

# ----------------------------------------------------------------------
# 3️⃣ get_asset 호출 및 델타/상태 필터링 (병렬 처리 실행)
# ----------------------------------------------------------------------
current_update_snapshot = {}
assets_to_save = []
MAX_THREADS = 50 

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    results = executor.map(fetch_and_filter_asset, all_glue_asset_ids)
    
    for asset in results:
        if not asset: continue
            
        unique_key = asset.get("unique_key")
        current_time_str = asset.get("current_update_time")
        
        if not unique_key or not current_time_str: continue 
            
        previous_time_str = previous_update_times.get(unique_key)

        # 💡 델타 판단 로직 (시나리오 1: 추가, 시나리오 2: 수정)
        if unique_key not in previous_update_times:
            # 1. 신규 생성 (추가)
            assets_to_save.append({"asset": asset, "action": "CREATED"})
        elif previous_time_str and is_time_newer(current_time_str, previous_time_str):
            # 2. 수정 (덮어쓰기)
            assets_to_save.append({"asset": asset, "action": "UPDATED"})
        
        # 다음 실행을 위해 현재 시간 스냅샷 기록 (ACTIVE 상태인 경우만 기록)
        if "metadata" in asset:
            current_update_snapshot[unique_key] = current_time_str

print(f"\n✅ Found {len(assets_to_save)} changes (CREATED/UPDATED) to synchronize.")

# ----------------------------------------------------------------------
# 4️⃣ S3/KB 폴더 동기화 로직 (3가지 시나리오 처리)
# ----------------------------------------------------------------------
previous_keys = set(previous_update_times.keys())
current_keys = set(current_update_snapshot.keys())

# 💡 4-A: PUBLISHED 해제/삭제된 Asset 감지 및 삭제 (시나리오 3)
keys_to_delete = previous_keys - current_keys

for unique_key in keys_to_delete:
    output_path = os.path.join(KB_S3_SYNC_DIR, f"{unique_key}.json")
    
    if os.path.exists(output_path):
        os.remove(output_path) # 로컬 파일 삭제 (S3 API 호출로 대체 필요)
        print(f"🗑️ Deleted UNPUBLISHED/DELETED Asset: {unique_key}")

# 💡 4-B: 신규 생성 및 변경된 Asset 저장 (시나리오 1 & 2)
for item in assets_to_save:
    asset = item["asset"]
    unique_key = asset["unique_key"]
    output_path = os.path.join(KB_S3_SYNC_DIR, f"{unique_key}.json") 

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(
            {"unique_key": unique_key, "metadata": asset["metadata"], "updatedAt": current_update_snapshot[unique_key]}, 
            f, default=str, ensure_ascii=False, indent=2
        )
    
    print(f"✅ KB File {item['action']}: {unique_key}.json")

# 💡 4-C: 현재 업데이트 시간 스ナップ샷 저장 (다음 실행을 위한 업데이트)
with open(PUBLISHED_ASSET_SNAPSHOT_FILE, "w") as f:
    json.dump(current_update_snapshot, f, indent=2)

print(f"\n🎉 Daily Delta Asset Management Complete. (Saved {len(assets_to_save)} changes, Deleted {len(keys_to_delete)} keys)")

Loaded 2 update records for delta comparison.

--- Step 2: Collecting ALL Asset IDs for Delta Check ---
✅ Total 50 assets found for detailed check.

✅ Found 1 changes (CREATED/UPDATED) to synchronize.
✅ KB File CREATED: default_cloudtrail_logs_mjkwon_smus_test.json

🎉 Daily Delta Asset Management Complete. (Saved 1 changes, Deleted 0 keys)
